## class State()

This class encodes the positions of the balls, dimensions of the table, and the state of the game.

balls is a dict of ball objects that must be initialized in __init__, sorted by their ID. This could be a tuple, but a dict means we can easy update/modify a specific ball (such as the cue ball)

team names are stored as strings

turn indicates which team is about is playing (1 or 2)

score indicates how many balls have been pocketed for each team [team 1 score, team 2 score]

In [ ]:
import numpy as np
import dataclasses


@dataclasses.dataclass
class State():
    balls: dict
    team_1 = str
    team_2 = str
    turn = int
    score = list[int, int]
    
    W_TABLE = 1.27 # meters
    H_TABLE = 2.54 # meters
    BALL_RADIUS = 0.06 # meters
    T = 0.01 # seconds
    
    def __init__(self, team_name_1: str, team_name_2: str):
        self.team_1 = team_name_1
        self.team_2 = team_name_2
        turn = 1
        score = [0,0]
        
        # make a whole bunch of balls and initial positions
        
    
    def update():
        
        "steps forward all balls and manages interactions"
        ...
        

# Questions to be discussed
globals...

In [ ]:
# Tests
